### Import Modules

In [ ]:
from python.read_pics import *
from os import listdir
from os.path import isfile,join,splitext
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy as sp
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
#from sklearn.linear_model import SGDClassifier
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.svm import LinearSVC
#from sklearn.neural_network import MLPClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import AdaBoostClassifier


### Getting files and pre-processing

In [ ]:
fichiers = [f for f in listdir("data/") if isfile(join("data/", f))]

In [ ]:
char_mapping = []
data = []
cpt = 0
for file in fichiers:
    char_mapping.append((splitext(file)[0])[5:])
    d, _ = get_pics_from_file("data/" + file)
    for i in range(len(d)):
        d[i] = np.append(d[i], cpt)
    cpt += 1
    data.append(d)
    
print(char_mapping)

In [ ]:
for t in range(len(data)):
    data[t] = data[t][0:6963]
    
data = np.array(data)
print(data.shape)

In [ ]:
data = data.reshape(42*6963, 18)
d = pd.DataFrame(data)
d.head()

In [ ]:
data.shape

### Split data set into a training set and a test one

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data[:, 0:17], data[:, 17], test_size=0.33)

In [ ]:
# Way of defining a good combinaisons of params for the EXTRATREECLASSIFIER algo.
'''
params = [{'min_samples_leaf': range(2,20,5),
          'min_samples_split': range(2,20,5),
         'criterion': ('gini', 'entropy')}]
clf = GridSearchCV(ExtraTreesClassifier(n_estimators=100), params)
clf.fit(X_train, y_train)
print(pd.DataFrame.from_dict(clf.cv_results_).loc[:,["params","mean_test_score"]])
clf.best_estimator_
print(clf.best_score_, clf.best_params_)
'''

### Training of the model.

In [ ]:
# Decrease n_estimators if no sufficient memory.
# The more n_estimators is high, the more the IA will be accurate.
clf = ExtraTreesClassifier(n_estimators=120)
clf.fit(X_train, y_train)
res = clf.predict(X_test)
accuracy_score(y_test, res)

### Detection of login/password

In [ ]:
def keylogger(): 
    frames, info = get_pics_from_file('pics_LOGINMDP.bin')
    histo = [0 for j in range(42)]
    result = []
    res = clf.predict(frames)
    res2 = []
    index = 0
    for c in res:
        index += 1
        histo[int(c)] += 1
        if index % 50 == 0:
            res2.append(char_mapping[np.argmax(histo)])
            histo = [0 for j in range(42)]
    return res2

keylogger()